##### import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [3]:
#one prompt for all tipes of code
import tatqa_utils
import pandas as pd

In [5]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [7]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [8]:
df = pd.read_json('0')

In [ ]:
def eval_res(res):
    good_type = 0
    for i in res:
       
       
        if (i["raw"]["answer_type"] == "arithmetic" and  i["answer2"] == "YES") or (i["raw"]["answer_type"] != "arithmetic" and  i["answer2"] == "NO"):
            good_type = good_type+1
        else:
            print("**", end="")        
            print(i["raw"]["answer_type"],"\t",i["answer2"],"\t", i["raw"]["answer_from"], "\t",i["answer"],"\t", i["question"])    
   
    print(good_type, len(res), good_type / len(res))
eval_res(res)


In [9]:
table = df.loc(0)[0]["table"]["table"]
text = ";".join([p["text"] for p in df.loc(0)[0]["paragraphs"]])
question = df.loc(0)[0]["questions"][0]["question"]
prompt = 4,
print(prompt)

Consider the following text and table as context. To answer the following question, is data from text, table, or both required? TEXT: Sales by Contract Type: Substantially all of our contracts are fixed-price type contracts. Sales included in Other contract types represent cost plus and time and material type contracts.;On a fixed-price type contract, we agree to perform the contractual statement of work for a predetermined sales price. On a cost-plus type contract, we are paid our allowable incurred costs plus a profit which can be fixed or variable depending on the contract’s fee arrangement up to predetermined funding levels determined by the customer. On a time-and-material type contract, we are paid on the basis of direct labor hours expended at specified fixed-price hourly rates (that include wages, overhead, allowable general and administrative expenses and profit) and materials at cost. The table below presents total net sales disaggregated by contract type (in millions): TABLE

In [16]:
res = []
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        prompt1 = f"Consider the following text and table as context. To answer the following question, is data from text, table, or both required? TEXT: {text} TABLE: {table} QUESTION: {question} Please answer with one word: TEXT, TABLE or BOTH. "
        answer = llm.invoke(prompt1).content
        prompt2 = f"Consider the following text and table as context. To answer the following question, is some arithmetic calculation required on data? TEXT: {text} TABLE: {table} QUESTION: {question} Please answer with one word: YES or NO."
        answer2 = llm.invoke(prompt2).content
        res.append({'raw': q, "question":question, "prompt": prompt, "answer": answer, "prompt2": prompt2, "answer2": answer2})
eval_res(res)
,

**span 	 YES 	 text 	 TEXT 	 What is the company paid on a cost-plus type contract?
**span 	 YES 	 table-text 	 TABLE 	 What is the amount of total sales in 2019?
**span 	 YES 	 table-text 	 TABLE 	 In which year is the amount of total sales the largest?
**multi-span 	 YES 	 table-text 	 TABLE 	 What are the types of Solutions segments in the table?
**span 	 YES 	 text 	 BOTH 	 How is the discount rate for domestic plans determined?
**span 	 YES 	 text 	 TEXT 	 How is the discount rate for international plans determined?
**span 	 YES 	 text 	 TEXT 	 How often does the company review the actuarial assumptions which the periodic benefit cost and the actuarial present value of projected benefit obligations are based on?
**multi-span 	 YES 	 table 	 TABLE 	 What financial items are listed in the table?
**span 	 YES 	 text 	 TEXT 	 Which countries does the group operate defined benefit schemes in?
**arithmetic 	 NO 	 table 	 TABLE 	 What is the 2019 average defined contribution schemes?
**a

In [15]:
res = []
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        prompt2 = f"Consider the following TEXT as context. To answer the QUESTION question, is some arithmetic calculation required on data? TEXT: {text} QUESTION: {question} Please answer with one word: YES or NO."
        answer2 = llm.invoke(prompt2).content
        res.append({'raw': q, "question":question, "prompt": prompt, "answer": answer, "prompt2": prompt2, "answer2": answer2})
eval_res(res)

**span 	 YES 	 text 	 TEXT 	 What is the company paid on a cost-plus type contract?
**span 	 YES 	 text 	 BOTH 	 How is the discount rate for domestic plans determined?
**span 	 YES 	 text 	 TEXT 	 How is the discount rate for international plans determined?
**span 	 YES 	 text 	 TEXT 	 How often does the company review the actuarial assumptions which the periodic benefit cost and the actuarial present value of projected benefit obligations are based on?
**arithmetic 	 NO 	 table 	 TABLE 	 What is the year on year percentage change in domestic discount rate between 2018 and 2019?
**arithmetic 	 NO 	 table 	 TABLE 	 What is the year on year percentage change in international expected return on plan assets between 2018 and 2019?
**span 	 YES 	 text 	 TEXT 	 Which countries does the group operate defined benefit schemes in?
**arithmetic 	 NO 	 table 	 TABLE 	 What is the 2019 average defined contribution schemes?
**arithmetic 	 NO 	 table 	 TABLE 	 What is the 2019 average defined benefit

In [ ]:
def eval_res(res):
    good_type = 0
    for i in res:
       
       
        if (i["raw"]["answer_type"] == "arithmetic" and  i["answer2"] == "YES") or (i["raw"]["answer_type"] != "arithmetic" and  i["answer2"] == "NO"):
            good_type = good_type+1
        else:
            print("**", end="")        
            print(i["raw"]["answer_type"],"\t",i["answer2"],"\t", i["raw"]["answer_from"], "\t",i["answer"],"\t", i["question"])
   
    print(good_type, len(res), good_type / len(res))
eval_res(res)

In [20]:
res = []
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        prompt2 = f"To answer the following question, is some arithmetic calculation required on data? QUESTION: {question} Please answer with one word: YES or NO."
        answer2 = llm.invoke(prompt2).content
        res.append({'raw': q, "question":question, "prompt": prompt, "answer": answer, "prompt2": prompt2, "answer2": answer2})
eval_res(res)

**span 	 YES 	 text 	 TABLE 	 What is the company paid on a cost-plus type contract?
**span 	 YES 	 table-text 	 TABLE 	 What is the amount of total sales in 2019?
**span 	 YES 	 table-text 	 TABLE 	 In which year is the amount of total sales the largest?
**multi-span 	 YES 	 table-text 	 TABLE 	 In which years was for the net sales by segment and industry end market calculated?
**span 	 YES 	 table 	 TABLE 	 In which year was the amount for Sensors the largest?
**span 	 YES 	 text 	 TABLE 	 How is the discount rate for domestic plans determined?
**span 	 YES 	 text 	 TABLE 	 How is the discount rate for international plans determined?
**span 	 YES 	 text 	 TABLE 	 How often does the company review the actuarial assumptions which the periodic benefit cost and the actuarial present value of projected benefit obligations are based on?
**multi-span 	 YES 	 table 	 TABLE 	 What financial items are listed in the table?
**span 	 YES 	 text 	 TABLE 	 Which countries does the group operate def

In [23]:
res = []
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        #prompt2 = f"To answer the following question, is some arithmetic calculation required on data? QUESTION: {question} Please answer with one word: YES or NO."
        prompt2 = f"To answer the following question, is arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: {question}"
        answer2 = llm.invoke(prompt2).content
        res.append({'raw': q, "question":question, "prompt": prompt, "answer": answer, "prompt2": prompt2, "answer2": answer2})
eval_res(res)


**span 	 YES 	 text 	 TABLE 	 What is the company paid on a cost-plus type contract?
**span 	 YES 	 table-text 	 TABLE 	 What is the amount of total sales in 2019?
**arithmetic 	 NO 	 table 	 TABLE 	 What is the 2019 average defined benefit schemes?
**multi-span 	 YES 	 table 	 TABLE 	 What was the operating loss carryforward amount in 2019 and 2018 respectively?
**multi-span 	 YES 	 table-text 	 TABLE 	 What was the net deferred tax asset before valuation allowance amount in 2019 and 2018 respectively?
**multi-span 	 YES 	 table 	 TABLE 	 What was the net deferred tax asset amount in 2019 and 2018 respectively?
24 30 0.8


In [24]:
res = []
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        #prompt2 = f"To answer the following question, is some arithmetic calculation required on data? QUESTION: {question} Please answer with one word: YES or NO."
        prompt2 = f"To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: {question}"
        answer2 = llm.invoke(prompt2).content
        res.append({'raw': q, "question":question, "prompt": prompt, "answer": answer, "prompt2": prompt2, "answer2": answer2})
eval_res(res)

**span 	 YES 	 text 	 TABLE 	 What is the company paid on a cost-plus type contract?
**span 	 YES 	 table-text 	 TABLE 	 What is the amount of total sales in 2019?
**span 	 YES 	 table-text 	 TABLE 	 In which year is the amount of total sales the largest?
**span 	 YES 	 table 	 TABLE 	 In which year was the amount for Sensors the largest?
**multi-span 	 YES 	 table 	 TABLE 	 What was the operating loss carryforward amount in 2019 and 2018 respectively?
**multi-span 	 YES 	 table-text 	 TABLE 	 What was the net deferred tax asset before valuation allowance amount in 2019 and 2018 respectively?
**multi-span 	 YES 	 table 	 TABLE 	 What was the net deferred tax asset amount in 2019 and 2018 respectively?
23 30 0.7666666666666667


In [ ]:
res = []
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        #prompt2 = f"To answer the following question, is some arithmetic calculation required on data? QUESTION: {question} Please answer with one word: YES or NO."
        prompt2 = f"To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: {question}"
        answer2 = llm.invoke(prompt2).content
        res.append({'raw': q, "question":question, "prompt": prompt, "answer": answer, "prompt2": prompt2, "answer2": answer2})
eval_res(res)

In [31]:
res = []
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        prompt2 = f"Consider the following text and table as context. To answer the following question, is some arithmetic calculation required on data, or can it be found in context without calculation? Please answer with one word: YES or NOTREQ. TEXT: {text} TABLE: {table} QUESTION: {question}"
        answer2 = llm.invoke(prompt2).content
        res.append({'raw': q, "question":question, "prompt": prompt, "answer": answer, "prompt2": prompt2, "answer2": answer2})
eval_res(res)


**span 	 NO 	 text 	 TABLE 	 What is the company paid on a cost-plus type contract?
**span 	 YES 	 table-text 	 TABLE 	 What is the amount of total sales in 2019?
**multi-span 	 NO 	 text 	 TABLE 	 What are the contract types?
**span 	 NO 	 table-text 	 TABLE 	 In which year is the amount of total sales the largest?
**arithmetic 	 NO 	 table-text 	 TABLE 	 What is the change in Other in 2019 from 2018?
**arithmetic 	 NO 	 table-text 	 TABLE 	 What is the percentage change in Other in 2019 from 2018?
**span 	 NO 	 text 	 TABLE 	 How is industry end market information presented?
**multi-span 	 NO 	 table-text 	 TABLE 	 In which years was for the net sales by segment and industry end market calculated?
**multi-span 	 YES 	 table-text 	 TABLE 	 What are the types of Solutions segments in the table?
**span 	 NO 	 table 	 TABLE 	 In which year was the amount for Sensors the largest?
**arithmetic 	 NO 	 table 	 TABLE 	 What was the change in the amount for Appliances in 2019 from 2018?
**arit

In [ ]:
# eval át lett írva NOTREQ-re

In [34]:
res = []
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        prompt2 = f"Consider the following text and table as context. To answer the following question, is some arithmetic calculation required on data, or can it be found in context without calculation? Please answer with one word: YES or NOTREQ. TEXT: {text} TABLE: {table} QUESTION: {question}"
        answer2 = llm.invoke(prompt2).content
        res.append({'raw': q, "question":question, "prompt": prompt, "answer": answer, "prompt2": prompt2, "answer2": answer2})
eval_res(res)


**span 	 YES 	 table-text 	 TABLE 	 What is the amount of total sales in 2019?
**span 	 YES 	 table-text 	 TABLE 	 In which year is the amount of total sales the largest?
**span 	 YES 	 text 	 TABLE 	 How often does the company review the actuarial assumptions which the periodic benefit cost and the actuarial present value of projected benefit obligations are based on?
**arithmetic 	 NOTREQ 	 table 	 TABLE 	 What is the 2019 average defined contribution schemes?
**arithmetic 	 NOTREQ 	 table 	 TABLE 	 What is the 2019 average defined benefit schemes?
**multi-span 	 YES 	 table 	 TABLE 	 What was the operating loss carryforward amount in 2019 and 2018 respectively?
**multi-span 	 YES 	 table-text 	 TABLE 	 What was the net deferred tax asset before valuation allowance amount in 2019 and 2018 respectively?
**multi-span 	 YES 	 table 	 TABLE 	 What was the net deferred tax asset amount in 2019 and 2018 respectively?
22 30 0.7333333333333333


In [37]:
def eval_res(res):
    good_type = 0
    for i in res:
       
       
        if (i["raw"]["answer_type"] == "arithmetic" and  i["answer2"] == "YES") or (i["raw"]["answer_type"] != "arithmetic" and  i["answer2"] == "NOTREQ"):
            good_type = good_type+1
        else:
            print("**", end="")        
            print(i["raw"]['uid'],i["raw"]["answer_type"],"\t",i["answer2"],"\t", i["raw"]["answer_from"], "\t",i["answer"],"\t", i["question"])
   
    print(good_type, len(res), good_type / len(res))
eval_res(res)

**4960801d-277d-4f79-8eca-c4d0200fa9d6 span 	 YES 	 table-text 	 TABLE 	 What is the amount of total sales in 2019?
**f4142349-eb72-49eb-9a76-f3ccb1010cbc span 	 YES 	 table-text 	 TABLE 	 In which year is the amount of total sales the largest?
**0f032004-ec01-40a0-831b-aac3f7e1b5c7 span 	 YES 	 text 	 TABLE 	 How often does the company review the actuarial assumptions which the periodic benefit cost and the actuarial present value of projected benefit obligations are based on?
**a0414f81-8dc2-44b2-a441-2c9d9c805c4d arithmetic 	 NOTREQ 	 table 	 TABLE 	 What is the 2019 average defined contribution schemes?
**bf7abd62-d9cd-48d2-8826-1457684019a3 arithmetic 	 NOTREQ 	 table 	 TABLE 	 What is the 2019 average defined benefit schemes?
**c3993366-da98-42f2-a5b2-3de28c4d0e10 multi-span 	 YES 	 table 	 TABLE 	 What was the operating loss carryforward amount in 2019 and 2018 respectively?
**5c7d8918-6083-4ac1-a0eb-137a114a7cd7 multi-span 	 YES 	 table-text 	 TABLE 	 What was the net deferred 